In [18]:
import pandas as pd
import spacy
import nltk
import re
from transformers import pipeline

In [9]:
def read_full_speech(filepath):
    with open(filepath, errors="ignore") as f:
        speech = f.readlines()
        speech = [s.replace("\n", "").replace(" | ", " ").split("|") for s in speech]#Full speech
        speech_df = pd.DataFrame(speech[1:], columns=speech[0])
    return speech_df

def read_speakermap(filepath):
    with open(filepath, errors="ignore") as f:
        speakermap_df = pd.read_table(f, delimiter = "|")
    return speakermap_df

def merge_speech_speaker(speech_df, speaker_df):
    speech_df = speech_df.astype({"speech_id": type(speaker_df.loc[:,'speech_id'][0])})
    return speaker_df.merge(speech_df, on="speech_id", how="left")

def get_speeches_filename(idx):
    return "speeches_{}.txt".format(idx)

def get_speakermap_filename(idx):
    return "{}_SpeakerMap.txt".format(idx)

def lemmatize(phrase):
    return " ".join([word.lemma_ for word in sp(phrase)])

def change_comma(speech):
    return re.sub("\.(?=\s[a-z0-9]|\sI[\W\s])", ",", speech)

In [3]:
filepath = "../hein-bound/{}".format(get_speeches_filename(111))
speech_df = read_full_speech(filepath)

filepath = "../hein-bound/{}".format(get_speakermap_filename(111))
speaker_df = read_speakermap(filepath)

final_df = merge_speech_speaker(speech_df, speaker_df)
final_df.head()

,speakerid,speech_id,lastname,firstname,chamber,state,gender,party,district,nonvoting,speech
0,111120160,1110000007,LARSON,JOHN,H,CT,M,D,1.0,voting,Our democracy renews itself every 2 years as M...
1,111117010,1110000009,PENCE,MIKE,H,IN,M,R,6.0,voting,Madam Clerk. as chairman of the Republican Con...
2,111118060,1110000013,BOEHNER,JOHN,H,OH,M,R,8.0,voting,Madam Speaker. Leader HOYER. fellow Members. a...
3,111120780,1110000014,PELOSI,NANCY,H,CA,F,D,8.0,voting,Thank you very much. Leader BOEHNER. Together....
4,111119830,1110000015,HOYER,STENY,H,MD,M,D,5.0,voting,Mr. CLYBURN. thank you.


In [4]:
df_merged = pd.concat([final_df, final_df.shift(-1).add_prefix('next_')], axis=1)
df_merged.head()

,speakerid,speech_id,lastname,firstname,chamber,state,gender,party,district,nonvoting,...,next_speech_id,next_lastname,next_firstname,next_chamber,next_state,next_gender,next_party,next_district,next_nonvoting,next_speech
0,111120160,1110000007,LARSON,JOHN,H,CT,M,D,1.0,voting,...,1.110000e+09,PENCE,MIKE,H,IN,M,R,6.0,voting,Madam Clerk. as chairman of the Republican Con...
1,111117010,1110000009,PENCE,MIKE,H,IN,M,R,6.0,voting,...,1.110000e+09,BOEHNER,JOHN,H,OH,M,R,8.0,voting,Madam Speaker. Leader HOYER. fellow Members. a...
2,111118060,1110000013,BOEHNER,JOHN,H,OH,M,R,8.0,voting,...,1.110000e+09,PELOSI,NANCY,H,CA,F,D,8.0,voting,Thank you very much. Leader BOEHNER. Together....
3,111120780,1110000014,PELOSI,NANCY,H,CA,F,D,8.0,voting,...,1.110000e+09,HOYER,STENY,H,MD,M,D,5.0,voting,Mr. CLYBURN. thank you.
4,111119830,1110000015,HOYER,STENY,H,MD,M,D,5.0,voting,...,1.110000e+09,LARSON,JOHN,H,CT,M,D,1.0,voting,for your nomination this morning. Thank you to...


In [5]:
speeches_with_questions = df_merged.loc[df_merged['speech'].str.contains('\?')]
speeches_with_questions[['speech', 'next_speech']].head(15)

,speech,next_speech
29,Will the gentlewoman yield?,I yield back the balance of my time.
36,Mr. Speaker. I thank the gentleman. Mr. Speake...,Mr. Speaker. I reserve my time.
39,I thank the gentleman very much. Mr. Speaker. ...,Mr. Speaker. I am pleased to yield 2 minutes t...
41,The gentleman from Wisconsin said. why will th...,Mr. Speaker. at this time I am happy to yield ...
43,Mr. Speaker. the Republican Members of the 111...,Mr. Speaker. please let me yield myself 1 or 2...
46,Mr. Speaker. may I inquire of the gentlewoman ...,I dont have any further requests for time. or ...
61,Mr. Speaker. I would just like to say that It ...,I am.
110,Mr. President. are we now in a period of morni...,Mr. President. on the Fourth of July of the ye...
125,Mr. President. I rise for a few moments to add...,Thank you. Mr. President. And I say to Senator...
126,Thank you. Mr. President. And I say to Senator...,Mr. President. would it be in order for me. be...


In [19]:
qa_pipeline = pipeline("question-answering")

In [34]:
for i in range(100):
    # change comma for both question speech and answer speech
    question = change_comma(speeches_with_questions.iloc[i].speech)
    answer = change_comma(speeches_with_questions.iloc[i].next_speech)
    # tokenize question speech
    quest_sents = nltk.sent_tokenize(question)
    print("Question: ", question)
    print("Answer: ", answer)
    # select question sentences from speech
    questions = [q for q in quest_sents if q.find("?")+1]
    print("Question sentences: ", questions)
    print()
    for q in questions:
        # get remaining speech after question
        pos = quest_sents.index(q)
        if pos + 1 < len(quest_sents):
            ans = ''.join(quest_sents[pos+1])
            question_df = pd.DataFrame.from_records([{
                'question': q,
                'context': context
            } for context in [ans, answer]])
        else:
            question_df = pd.DataFrame.from_records([{
                'question': q,
                'context': context
            } for context in [answer]])
        print("Question: ", q)
        preds = qa_pipeline(question_df.to_dict('records'))
        print(preds)
        print()
    print("###############################")

Question:  Will the gentlewoman yield?
Answer:  I yield back the balance of my time.
Question sentences:  ['Will the gentlewoman yield?']

Question:  Will the gentlewoman yield?
{'score': 0.19971067792387842, 'start': 2, 'end': 35, 'answer': 'yield back the balance of my time.'}

###############################
Question:  Mr. Speaker, I thank the gentleman. Mr. Speaker, you dont have to look far to see that families across this country are gripped with a tremendous amount of fear and uncertainty. They fear for their jobs, if they have one. They fear for their future as they see their 401(k)s, their college savings accounts collapse. They fear that their elected leaders dont get it. They fear that this Congress may very well be incapable of change, incapable of producing the kind of results that they want and to get it right. Under existing House rules, when a bill is brought to the floor that includes a tax increase, the minority has a right to offer a motion to strike that increase, a

[{'score': 0.261390552700135, 'start': 0, 'end': 17, 'answer': 'Thats the problem.'}, {'score': 0.103228894568681, 'start': 26, 'end': 55, 'answer': 'I am happy to yield 2 minutes'}]

###############################
Question:  Mr. Speaker, the Republican Members of the 111th Congress collectively represent more than 100 million constituents in this Nation. The changes that are being contemplated by the majority today represent an erosion, not of the interests of elected officials, not even of the interests of a political party, but. Mr. Speaker, I say with respect, it represents an erosion of the interests represented in this place of over 100 million Americans. As I listen to this debate, I cant help but wonder what our constituents who might be looking down from the gallery and looking in from elsewhere are thinking. How does this affect them? Instructions being promptly or forthwith, motions to recommit. But really what we are here to object to in this rule package is really the dea

{'score': 0.1026129975043153, 'start': 49, 'end': 54, 'answer': '1851,'}

###############################
Question:  Mr. President, I rise for a few moments to address two subjects, the first will be about Israel and the second about the passing of Griffin Bell. All of us are deeply concerned with the conditions in the Middle East, most recently in the last 12 days, the actions in Gaza, the loss of human life and the conflict. But there is a necessary perspective we all must understand. In November of 2007, I stood at the last Israeli outpost overlooking Gaza. In fact, if you watch Fox or CNN or NBC or ABC tonight, where you will see those reports coming from, I stood on that very spot just a little over a year ago. Also, I went to Sderot, the Israeli settlement outside Gaza, that since mid year last year has received 1, 2, 3, 10, 15 missile attacks, random attacks coming out of Gaza dropping on this Israeli settlement for no reason at all but the absolute ability or desire to terroriz

[{'score': 0.2224421456958794, 'start': 21, 'end': 96, 'answer': 'react to protect our citizens and protect their lives and their livelihoods.'}, {'score': 0.3286536603148633, 'start': 11039, 'end': 11097, 'answer': 'One would think those answers would be lethal politically,'}]

Question:  During Olympics, when they came to Atlanta in 1996 and there were difficulties, to whom did the Olympic committee go to weed through the minefield of Washington to get the security assistance necessary for the Olympics and Atlanta?
[{'score': 0.20688511799988163, 'start': 7, 'end': 19, 'answer': 'Griffin Bell.'}, {'score': 4.001486487023153e-05, 'start': 7909, 'end': 7959, 'answer': 'He was very disabled, and he came in a wheelchair,'}]

Question:  When there was a company that was in need of a forensic audit by a legal man who would come in and clean up a problem in their company, such as E.F. Hutton did, whom did they call?
[{'score': 0.7089313171250211, 'start': 12, 'end': 24, 'answer': 'Griffin B

[{'score': 0.5705800936420786, 'start': 214, 'end': 228, 'answer': 'most Americans.'}, {'score': 0.03944313732944105, 'start': 0, 'end': 14, 'answer': 'Mr. President,'}]

###############################
Question:  Mr. President, would it be in order for me, before I begin my remarks, to compliment the Presiding Officer for his nomination to be Cabinet Secretary, the Secretary of the Interior, and wish him very well before the Senate in being confirmed and serving in that position? I guess that question doesnt need a response, I certainly hope it is in line for me to be able to say that.
Answer:  Mr. President, I hopeand I am joined here by Senator LIEBERMANthat the Senate will have an opportunity to consider before this week is out a resolution we believe has been drafted by the majority leader and the minority leader that deals with the ongoing war in the Gaza Strip and that we believe needs to express the will of the Senate. We believe as well that a similar resolution would be voted

...

[{'score': 0.865593877530312, 'start': 4, 'end': 9, 'answer': 'theft.'}, {'score': 0.017435835748792483, 'start': 1881, 'end': 1934, 'answer': 'to end a situation where 1 million Israelis every day'}]

Question:  Now, I was really amazed when some of us, a bipartisan group of Members of Congress, went to China a few years ago, and talking to CEOs and since then talking to other CEOs, why was your industry moved to China?
[{'score': 0.06972346867587476, 'start': 43, 'end': 57, 'answer': 'cheaper labor,'}, {'score': 0.0672632470433765, 'start': 29705, 'end': 29767, 'answer': 'he has not spent a single penny buying bad bonds from anybody.'}]

Question:  Then we were told by some authorities that we had too much money in savings, that we either had to buy stuff or give it away or spend it somehow, get rid of it, or wed have to move out of Federal housing, I mean, whats wrong with this?
[{'score': 0.07420551793425734, 'start': 146, 'end': 196, 'answer': 'we lure people into a rut and we don

[{'score': 0.06380270934119281, 'start': 11, 'end': 33, 'answer': 'do to cause it to stop?'}, {'score': 0.8108079751950044, 'start': 7518, 'end': 7524, 'answer': 'Hamas,'}]

Question:  What do we do to cause it to stop?
[{'score': 0.2850131714912063, 'start': 41, 'end': 82, 'answer': 'the press has a remarkably silly approach.'}, {'score': 0.20018932862464922, 'start': 3512, 'end': 3550, 'answer': 'terrorizing of its innocent civilians.'}]

Question:  Well, what is the objective that Israel is seeking?
[{'score': 0.19383778050031886, 'start': 37, 'end': 55, 'answer': 'to end a situation'}, {'score': 0.8981304261077696, 'start': 7847, 'end': 7857, 'answer': 'happiness,'}]

Question:  Why do we pick 2005?
[{'score': 0.03522308654567485, 'start': 38, 'end': 121, 'answer': 'Israel withdrew completely, unilaterally, without concession, without compensation,'}, {'score': 0.1755640880926208, 'start': 2135, 'end': 2168, 'answer': 'disengagement from the Gaza Strip'}]

Question:  So what did Is

[{'score': 0.02880444395209114, 'start': 129, 'end': 151, 'answer': '1 weekjust 1 weekunder'}, {'score': 0.26506922390021614, 'start': 27, 'end': 51, 'answer': 'the House do now adjourn.'}]

###############################
Question:  Mr. President, I appreciate the cooperation and willingness to work with me of the Senator from New Mexico. He has been a gentleman throughout. We have always had conversations, we just havent agreed on what we have done. It has been a pleasure to work with him. Here is a 1.300page bill. People are going to say a lot of this has been around for a long time, that it doesnt need any debate, that it certainly should not be amended, but it is 1.300 pages. The CBO has refused to score this one. The last one they scored was between $6 and $8 billion. This is somewhere between $10 and $12 billion, especially when -we take the outyears beyond 10 years out of it. So here we sit with a 1.300page bill that has 45 blatant earmarks in it with no ability to amend. Since

[{'score': 0.07167841754549009, 'start': 0, 'end': 22, 'answer': 'A lot of my colleagues'}, {'score': 0.03685110138880621, 'start': 261, 'end': 329, 'answer': 'reservations about supporting its final passage in its current form.'}]

Question:  The question the American people ought to be asking is, with 165 bills, 1.300 pages, is now the time for us to set in motion to take an additional 2.2 million acres out of energy production and limit energy exposure to about 5 or 6 million more acres, and raise the total number of wilderness acres to 2 million greater than that we have in total development in the country?
[{'score': 0.00016178807016327922, 'start': 0, 'end': 45, 'answer': 'How long ago was it we had $4agallon gasoline?'}, {'score': 4.6688876122402895e-06, 'start': 123, 'end': 167, 'answer': 'Omnibus Public Lands Management Act of 2009.'}]

Question:  How long ago was it we had $4agallon gasoline?
[{'score': 0.3035432377294871, 'start': 0, 'end': 42, 'answer': 'Do we not think tha

{'score': 0.2796652447706016, 'start': 88, 'end': 98, 'answer': '11 minutes'}

###############################
Question:  Thank you. Mr. Chairman. My comments will be brief, I, too, rise today to speak in favor of cloture on the motion to proceed to S, 22, the Omnibus Public Land Management Act of 2009. The omnibus bill has been criticized as being largeand it is a large pile of paper. It is almost 1.300 pages. We acknowledge that. But this package of bills before us today also represents a huge commitment of time, a large commitment of resources by the Committee on Energy and Natural Resources, as well as the other four Senate committees. In the case of the Energy Committee, this package, along with a similar package that was passed by the Senate last spring, represents almost 2 years worth of hearings, negotiations, and business meetings on all of these public lands issues. This package contains over 160 public lands bills, the vast majority of which went through the regular committe